In [10]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import numpy as np

Using TensorFlow backend.


In [27]:
def vgg19Model(image_size = (224, 224), if_draw_model = False):
    width = image_size[0]
    height = image_size[1]
    
    base_model = ResNet50(input_tensor=Input((height, width, 3)), weights='imagenet', include_top=False)

    for layers in base_model.layers:
        layers.trainable = False
        
 #   for layer in model.layers[140:]:
 #       layer.trainable = True

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(10, activation='sigmoid')(x)
    model = Model(base_model.input, x)
    
    if if_draw_model:
        SVG(model_to_dot(model).create(prog='dot', format='svg'))
    
    model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model
    

In [20]:
import os
import shutil
from sklearn.utils import shuffle
np.random.seed(2017)

def prepare_val():
    dirname = 'imgs/val'
    dirname_src = 'imgs/train/'
    if os.path.exists(dirname):
        return
    os.mkdir(dirname)
    sub_dirs = ['c0', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9']
    for sub_dir in sub_dirs:
        address_des = dirname + '/' + sub_dir
        os.mkdir(address_des)
        
        address_src = dirname_src + '/' + sub_dir
        train_filenames = os.listdir(address_src)
        train_filenames = shuffle(train_filenames)
        for file_src in train_filenames[0: int(len(train_filenames) / 10)]:
            add_con_list = file_src.split('/')
            add_old = dirname_src + sub_dir + '/' + add_con_list[-1]
            add_new = dirname + '/' + sub_dir + '/' + add_con_list[-1]
         #   print(add_old)
         #   print(add_new)
            shutil.move(add_old, add_new)  
        
prepare_val()
    

In [25]:
#训练
nb_train_samples = 20197
def train_model(Model, image_size=(224, 224),batch_size =64):
    model = Model()
    
    train_datagen = ImageDataGenerator(
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
    
    train_generator = train_datagen.flow_from_directory(
    'imgs/train',
    target_size=(image_size[0], image_size[1]),
    batch_size=batch_size,
    class_mode='categorical')
    
    val_generator = train_datagen.flow_from_directory(
    'imgs/val',
    target_size=(image_size[0], image_size[1]),
    batch_size=batch_size,
    class_mode='categorical')
        
    model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=10,
    validation_data = val_generator)
    
    model.save_weights('model.h5')
    
    return model


In [28]:
train_model(vgg19Model)

Found 20187 images belonging to 10 classes.
Found 2237 images belonging to 10 classes.
Epoch 1/10
  2/315 [..............................] - ETA: 2:56:40 - loss: 2.4724 - acc: 0.1016

KeyboardInterrupt: 